<h1 align="center">Side-by-Side</h1>

# План
1. Пререквизиты
2. Pairwise vs Pointwise vs Listwise подход
3. Простой SBS: сравнение двух экранов
4. SBS с таргетированием
5. SBS-ранжирование нескольких картинок
6. SBS с поиском.
7. (*) Вебвизор

# Пререквизиты:

1. Необходимо [зарегистрироваться](https://toloka.yandex.ru/for-requesters/) в Толоке как заказчик
2. [Получить OAuth-токен](https://toloka.yandex.ru/requester/profile/integration) по ссылке. Подробнее в [документации](https://yandex.ru/dev/toloka/doc/concepts/access-docpage/) 
3. Разобраться со одним из способов хранения файлов для заданий



# Куда складывать файлы?
1. Использовать произвольное постоянное S3–хранилище файлов. Например, [Yandex.Cloud.Object.Storage](https://cloud.yandex.ru/docs/storage/quickstart?utm_source=console&utm_medium=empty-page&utm_campaign=storage)<br>
<span style="color:green">Подходит, если планируете ставить разметки на большой поток</span>


2. Использовать встроенный в Толоку механизм [Яндекс.Диска](https://yandex.ru/support/toloka-requester/concepts/prepare-data.html)<br>
<span style="color:green">Подходит для разовых запусков</span>


3. Использовать произвольное хранилище, которое позволяет получить URL на картинку (графический объект) вида: https://sbs.s3.yandex.net/39a307e3f4859c96f37161b3ab00aa5daa99858fbd6df1b70f53fa9a649ea467.png

*Замечание: весь написанный код должен воспроизвестись, если указать свой токен заказчика. Тем не менее, что-то может устареть и потребовать доработок.*

In [1]:
import requests
import scipy.stats
import time
import pandas as pd
import json
from IPython.core.display import display, HTML
from toloka_api import *

from pathlib import Path 
home = str(Path.home())

#Класс для работы с Толокой и функции для агрегации результатов
from toloka_api import TolokaProcessingApi
from toloka_results_aggregation import prepare_toloka_task_for_2_screens, prepare_toloka_task_for_many_screens

#Считываем токен Толоки из файлика.
toloka_token = open(home + "/.tokens/toloka").read().strip()

Класс для работы с API толоки

In [2]:
toloka_api = TolokaProcessingApi(toloka_token)

# Pointwise vs Pairwise vs Listwise?

* Поточечный подход (pointwise)
* Попарный подход (pairwise) 
* Списочный подход (listwise)






Подробнее можно прочитать в источниках:
* https://medium.com/@nikhilbd/pointwise-vs-pairwise-vs-listwise-learning-to-rank-80a8fe8fadfd
* http://www.machinelearning.ru/wiki/images/8/89/Voron-ML-Ranking-slides.pdf


<img src="https://sbs.s3.yandex.net/d7e825321372f31fd8761224fa572d379e1f218d3ffd512a17e588257a293478.jpg" width="400">


# Примеры решаемых задач с помощью Side-by-Side

1. Сравнение поисковых выдач
2. Сравнение интерфейсных решений
3. Выбор иконок
4. Ранжирование картинок (например, фонов)
5. Сравнение видео

# Настраиваем шаблон (проект и пул) для сравнения картинок

В Толоке есть стандартный шаблон для парных сравнений:

**Шаг 1.** Создаем новый проект.  https://toloka.yandex.ru/requester/templates

**Шаг 2.** Выбираем шаблон *парное сравнение изображений*
![](https://sbs.s3.yandex.net/16546bf0f93cfb664c4ceb2da7048fecfe734ed2833deb1986949cdc591d0322.png)


**Шаг 3.** Заполняем название, описание, инструкцию
![](https://sbs.s3.yandex.net/c9689c2359f2990bb432cc9ff4d4aa186ddd83addb605c8d509a299a7d424c61.png)

**Шаг 4.** Смотрим предпросмотр – как это все будет выглядеть для __респондентов__
![](https://sbs.s3.yandex.net/579e97f314fc945c323d0a9bee9784feea45b6d439992bed44e0da6f2b20834b.png)


**Шаг 5.** Чего не хватает?
1. У нас не "дубли/не дубли", а выбор Left/Right
2. Описания комментария/задания вверху страницы
3. Текстового ответа: чтобы респондент смог ответить на вопрос комментарием

**Шаг 6.** Создадим проект и добавим того, чего не хватает
![](https://sbs.s3.yandex.net/81441b92db3d1216881dcfbca2f17249cdb244cf29c34570df659088cdc3627a.png)

**Шаг 7.** Получается вот такой опросник для респондента
![](https://sbs.s3.yandex.net/45cf35af758525beff86d3370f5967aa95a298f22553b016d269386b90171ee8.png)

**Шаг 8.** Заводим "эталонный пул" с настройками
![](https://sbs.s3.yandex.net/aeb5998ba0eaca6d087ff1e7b9eb7ff3ad1fdb9fde11b8102e4a5aa54c6c5088.png)

__Замечания:__ 
1. Точно такой проект можно создать с помощью API, что и будет сделано далее
2. Шаблон можно совершенствовать и полировать до бесконечности

Заведем такой проект через [API Толоки](https://yandex.ru/dev/toloka/doc/concepts/create-prj-docpage/).

Параметры проекта из примера сложены в файл `project_templates/pictures_comparison_project_template.json`

In [3]:
project_template = json.loads(open("project_templates/pictures_comparison_project_template.json").read())
project_id = toloka_api.create_toloka_project(project_template)


New project was created. New project id:  35047
https://toloka.yandex.ru/requester/project/35047


Заведем шаблонный пул, с которого впоследствии будем клонировать с помощью [API Толоки](https://yandex.ru/dev/toloka/doc/concepts/create-pool-docpage/) 


Парамеры шаблонного пула лежат в файле `project_templates/pool_params_template.json`


In [5]:
pool_template = json.loads(open("project_templates/pool_params_template.json").read())
template_pool_id = toloka_api.create_toloka_pool(project_id, pool_template)

New pool was created. New pool id:  12965550




Хотим решить, какая плитка больше нравится людям

<table>
  <tr>
    <td><img src="https://sbs.s3.yandex.net/44aebb8c77a29efa25e8a5d1ac4eba9877b84efa1147b4b003b2a8380273cc73.png" width="250"></td>
      <td>  </td>
     <td><img src="https://sbs.s3.yandex.net/93656faf648029e3bf4bf18421b3dc08d85fe4bdd2683498cbf497f0487e89db.png" width="250"></td>
  </tr>
</table>

Подготовим задания для сравнения двух картинок

In [6]:
first_url = "https://sbs.s3.yandex.net/44aebb8c77a29efa25e8a5d1ac4eba9877b84efa1147b4b003b2a8380273cc73.png"
second_url = "https://sbs.s3.yandex.net/93656faf648029e3bf4bf18421b3dc08d85fe4bdd2683498cbf497f0487e89db.png"

print(first_url)
print(second_url)

https://sbs.s3.yandex.net/44aebb8c77a29efa25e8a5d1ac4eba9877b84efa1147b4b003b2a8380273cc73.png
https://sbs.s3.yandex.net/93656faf648029e3bf4bf18421b3dc08d85fe4bdd2683498cbf497f0487e89db.png


In [10]:
two_pictures_pool_id = toloka_api.clone_toloka_pool(template_pool_id)
tasks = prepare_toloka_task_for_2_screens(first_url, second_url, 
                                         50, two_pictures_pool_id)


Pool was cloned. New pool was created. New pool id:  12965570
https://toloka.yandex.ru/requester/pool/12965570


Далее добавим их в наш пул. 

In [12]:
toloka_api.upload_toloka_tasks(tasks)

Получили новый пул с залитыми заданиями. Теперь запустим его

In [14]:
toloka_api.print_toloka_pool_link(two_pictures_pool_id)
toloka_api.run_toloka_pool(two_pictures_pool_id)

https://toloka.yandex.ru/requester/pool/12965570
Pool 12965570 was started


Пул уехал в обработку респонентами. 
Можно пойти пить кофе, пока пул обрабатывается

Когда пул разметился, нужно выгрузить результаты и обработать их

In [3]:
d = toloka_api.get_pool_completed_tasks(two_pictures_pool_id)

In [4]:
d.head()

,question,image_left,image_right,result,comment,winner_url,loser_url
0,Какой вариант вам больше нравится?,https://sbs.s3.yandex.net/93656faf648029e3bf4b...,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,LEFT,на правом много красного\nне прикольно,https://sbs.s3.yandex.net/93656faf648029e3bf4b...,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...
1,Какой вариант вам больше нравится?,https://sbs.s3.yandex.net/93656faf648029e3bf4b...,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,RIGHT,белый текст на красном фоне более читаемый\n,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,https://sbs.s3.yandex.net/93656faf648029e3bf4b...
2,Какой вариант вам больше нравится?,https://sbs.s3.yandex.net/93656faf648029e3bf4b...,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,RIGHT,NaN,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,https://sbs.s3.yandex.net/93656faf648029e3bf4b...
3,Какой вариант вам больше нравится?,https://sbs.s3.yandex.net/93656faf648029e3bf4b...,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,RIGHT,Нечеткое изображение,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,https://sbs.s3.yandex.net/93656faf648029e3bf4b...
4,Какой вариант вам больше нравится?,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,https://sbs.s3.yandex.net/93656faf648029e3bf4b...,LEFT,NaN,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,https://sbs.s3.yandex.net/93656faf648029e3bf4b...


Число людей, у которых:
* слева first_url
* выбрали first_url

In [7]:
d[(d['image_left'] == first_url) & (d['result'] == "LEFT")].shape[0]

18

Число людей, у которых:
* слева second_url
* выбрали second_url

In [8]:
d[(d['image_left'] == second_url) & (d['result'] == "LEFT")].shape[0]

8

Число людей, у которых:
* слева first_url
* выбрали second_url

In [9]:
d[(d['image_left'] == first_url) & (d['result'] == "RIGHT")].shape[0]

7

Число людей, у которых:
* слева second_url
* выбрали first_url

In [10]:
d[(d['image_left'] == second_url) & (d['result'] == "RIGHT")].shape[0]

17

Для удобства сделаем колоночки winner_url (выбранный URL) и loser_url (проигравший URL)

In [11]:
d.head()

,question,image_left,image_right,result,comment,winner_url,loser_url
0,Какой вариант вам больше нравится?,https://sbs.s3.yandex.net/93656faf648029e3bf4b...,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,LEFT,на правом много красного\nне прикольно,https://sbs.s3.yandex.net/93656faf648029e3bf4b...,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...
1,Какой вариант вам больше нравится?,https://sbs.s3.yandex.net/93656faf648029e3bf4b...,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,RIGHT,белый текст на красном фоне более читаемый\n,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,https://sbs.s3.yandex.net/93656faf648029e3bf4b...
2,Какой вариант вам больше нравится?,https://sbs.s3.yandex.net/93656faf648029e3bf4b...,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,RIGHT,NaN,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,https://sbs.s3.yandex.net/93656faf648029e3bf4b...
3,Какой вариант вам больше нравится?,https://sbs.s3.yandex.net/93656faf648029e3bf4b...,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,RIGHT,Нечеткое изображение,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,https://sbs.s3.yandex.net/93656faf648029e3bf4b...
4,Какой вариант вам больше нравится?,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,https://sbs.s3.yandex.net/93656faf648029e3bf4b...,LEFT,NaN,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,https://sbs.s3.yandex.net/93656faf648029e3bf4b...


In [19]:
d['winner_url'].value_counts() 

https://sbs.s3.yandex.net/44aebb8c77a29efa25e8a5d1ac4eba9877b84efa1147b4b003b2a8380273cc73.png    35
https://sbs.s3.yandex.net/93656faf648029e3bf4bf18421b3dc08d85fe4bdd2683498cbf497f0487e89db.png    15
Name: winner_url, dtype: int64

Надо посчитать стат.значимость. Сделать это можно с помощью биномиального теста.

https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.stats.binom_test.html

In [20]:
scipy.stats.binom_test(35, 50)

0.006600447966810915

Можно сделать вывод о том, что различие значимое.

Еще могут быть полезными комментарии

In [30]:
d[(d['winner_url'] == first_url)].dropna().head()

,question,image_left,image_right,result,comment,winner_url,loser_url
1,Какой вариант вам больше нравится?,https://sbs.s3.yandex.net/93656faf648029e3bf4b...,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,RIGHT,белый текст на красном фоне более читаемый\n,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,https://sbs.s3.yandex.net/93656faf648029e3bf4b...
3,Какой вариант вам больше нравится?,https://sbs.s3.yandex.net/93656faf648029e3bf4b...,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,RIGHT,Нечеткое изображение,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,https://sbs.s3.yandex.net/93656faf648029e3bf4b...
6,Какой вариант вам больше нравится?,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,https://sbs.s3.yandex.net/93656faf648029e3bf4b...,LEFT,Нравиться больше так как красный фон,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,https://sbs.s3.yandex.net/93656faf648029e3bf4b...
9,Какой вариант вам больше нравится?,https://sbs.s3.yandex.net/93656faf648029e3bf4b...,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,RIGHT,единый стиль,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,https://sbs.s3.yandex.net/93656faf648029e3bf4b...
10,Какой вариант вам больше нравится?,https://sbs.s3.yandex.net/93656faf648029e3bf4b...,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,RIGHT,более контрасно,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,https://sbs.s3.yandex.net/93656faf648029e3bf4b...


In [25]:
d[(d['winner_url'] == second_url)].dropna().head()

,question,image_left,image_right,result,comment,winner_url,loser_url
0,Какой вариант вам больше нравится?,https://sbs.s3.yandex.net/93656faf648029e3bf4b...,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,LEFT,на правом много красного\nне прикольно,https://sbs.s3.yandex.net/93656faf648029e3bf4b...,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...
24,Какой вариант вам больше нравится?,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,https://sbs.s3.yandex.net/93656faf648029e3bf4b...,RIGHT,не так ярко красно,https://sbs.s3.yandex.net/93656faf648029e3bf4b...,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...
25,Какой вариант вам больше нравится?,https://sbs.s3.yandex.net/93656faf648029e3bf4b...,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,LEFT,Более привлекательно выглядит,https://sbs.s3.yandex.net/93656faf648029e3bf4b...,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...
30,Какой вариант вам больше нравится?,https://sbs.s3.yandex.net/93656faf648029e3bf4b...,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,LEFT,красный цвет збирает на ебя все внимание а бел...,https://sbs.s3.yandex.net/93656faf648029e3bf4b...,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...
31,Какой вариант вам больше нравится?,https://sbs.s3.yandex.net/93656faf648029e3bf4b...,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...,LEFT,"Не такая яркая, не режет глаза.",https://sbs.s3.yandex.net/93656faf648029e3bf4b...,https://sbs.s3.yandex.net/44aebb8c77a29efa25e8...


# Задача: отсортировать множество объектов: например, фонов

Есть несколько картинок:


<table>
  <tr>
    <td><img src="https://sbs.s3.yandex.net/86b63f8f20396ec95cbbd0b1b9c5516dc294e6142e31b96c849101f7de36966b.png" width="450"></td>
      <td>  </td>
     <td><img src="https://sbs.s3.yandex.net/d6c07660358fa23737c6b8845f7e1320904bbdb665cda19550d898848378a96f.png" width="450"></td>
      <td>  </td>
     <td><img src="https://sbs.s3.yandex.net/002fbd80aeebb07dbb68a363a986c06b4f3fbb4bbe6644822bb038856d686aaf.png" width="450"></td>
  </tr>
    
  <tr>
    <td><img src="https://sbs.s3.yandex.net/df1b5c6f2bb11149b776e896c49d9a04353a1d72c2c72e122bf2f462982e9b23.png" width="450"></td>
      <td>  </td>
     <td><img src="https://sbs.s3.yandex.net/babf0f5ffd8bb0ef6762ceef89994b93f3df37515cd74c2c2bb7c50c87e2d1e6.png" width="450"></td>
      <td>  </td>
     <td><img src="https://sbs.s3.yandex.net/be1bffed70943f42c2f607292e76dfb96361a33f37824bca22223d29092402b6.png" width="450"></td>
  </tr>
</table>

И не можем решить, что лучше использовать в качестве фона рабочего стола.

Проведем множество парных сравнений и опросим...респондентов!

In [466]:
pictures_urls = ["https://sbs.s3.yandex.net/86b63f8f20396ec95cbbd0b1b9c5516dc294e6142e31b96c849101f7de36966b.png",
                 "https://sbs.s3.yandex.net/d6c07660358fa23737c6b8845f7e1320904bbdb665cda19550d898848378a96f.png",
                 "https://sbs.s3.yandex.net/002fbd80aeebb07dbb68a363a986c06b4f3fbb4bbe6644822bb038856d686aaf.png",
                 "https://sbs.s3.yandex.net/df1b5c6f2bb11149b776e896c49d9a04353a1d72c2c72e122bf2f462982e9b23.png",
                 "https://sbs.s3.yandex.net/babf0f5ffd8bb0ef6762ceef89994b93f3df37515cd74c2c2bb7c50c87e2d1e6.png" ,
                 "https://sbs.s3.yandex.net/be1bffed70943f42c2f607292e76dfb96361a33f37824bca22223d29092402b6.png"]

Заводим очередной пул

In [476]:
pictures_pool = clone_toloka_pool(template_pool_id)
tasks = prepare_toloka_task_for_many_screens(pictures_urls, 5, 
                                          pictures_pool, "Какой фон для рабочего стола вам больше нравится?")

upload_toloka_tasks(tasks)

print_toloka_pool_link(pictures_pool)

Pool was cloned. New pool was created. New pool id:  8809616
https://toloka.yandex.ru/requester/pool/8809616


Проверяем, запускаем и снова идем пить кофе!

In [477]:
run_toloka_pool(pictures_pool)

Pool 8809616 was started


In [536]:
pictures_results = pd.DataFrame(get_pool_completed_tasks(pictures_pool)["winner_url"].value_counts()).reset_index()
pictures_results["winner_url"] = pictures_results["winner_url"] / sum(pictures_results["winner_url"])
pictures_results.columns = ["url", "rate"]
pictures_results["url"] = pictures_results["url"].apply(
                        lambda x: '<a href="{}"><img src="{}" width=300px></a>'.format(x, x))

pd.options.display.max_colwidth = -1
html_all = pictures_results.to_html(escape=False, )
display(HTML(html_all))
pd.options.display.max_colwidth = 50


,url,rate
0,,0.280000
1,,0.226667
2,,0.226667
3,,0.133333
4,,0.106667
5,,0.026667


# Пример с таргетированием

Часто нужно как-то отсечь аудиторию. 

Рассмотрим пример: запустим эксперимент сначала на аудитории из Москвы, а потом на аудитории из Санкт-Петербурга

<table>
  <tr>
    <td><img src="https://sbs.s3.yandex.net/af24437833aef6d80f05db322f441cd0573f3c886ccc692d68d410180de481b6.png" width="450"></td>
      <td>  </td>
     <td><img src="https://sbs.s3.yandex.net/35358c7261be07d066c4eb06451a83ed3049721b8a8ae8e50d1b4a0d909e7aec.png" width="450"></td>
  </tr>
</table>

Готовим 2 пула: "Московский" и "Питерский"

In [422]:
first_url = "https://sbs.s3.yandex.net/af24437833aef6d80f05db322f441cd0573f3c886ccc692d68d410180de481b6.png" 
second_url = "https://sbs.s3.yandex.net/35358c7261be07d066c4eb06451a83ed3049721b8a8ae8e50d1b4a0d909e7aec.png" 

In [441]:
msk_pool = clone_toloka_pool(template_pool_id)
tasks = prepare_toloka_task_for_2_screens(first_url, second_url, 
                                         50, msk_pool)

upload_toloka_tasks(tasks)

Pool was cloned. New pool was created. New pool id:  8808652


In [442]:
spb_pool = clone_toloka_pool(template_pool_id)
tasks = prepare_toloka_task_for_2_screens(first_url, second_url, 
                                         50, spb_pool)

upload_toloka_tasks(tasks)

Pool was cloned. New pool was created. New pool id:  8808656


In [443]:
msk_pool_params=get_pool_params(msk_pool)
msk_target_filter = {'or': [{'category': 'profile',
      'key': 'city',
      'operator': 'IN',
      'value': 213}]}

user_filter = msk_pool_params['filter']['and']
user_filter.append(msk_target_filter)
msk_pool_params['filter']['and'] = user_filter

update_pool_params(msk_pool, msk_pool_params)

In [451]:
spb_pool_params=get_pool_params(spb_pool)
spb_target_filter = {'or': [{'category': 'profile',
      'key': 'city',
      'operator': 'IN',
      'value': 2}]}

user_filter = spb_pool_params['filter']['and']
user_filter.append(spb_target_filter)
spb_pool_params['filter']['and'] = user_filter

update_pool_params(spb_pool, spb_pool_params)

Проверим в интерфейсе, что настройки правильно доехали

In [447]:
print_toloka_pool_link(msk_pool)
print_toloka_pool_link(spb_pool)

https://toloka.yandex.ru/requester/pool/8808652
https://toloka.yandex.ru/requester/pool/8808656


<table>
  <tr>
    <td><img src="https://sbs.s3.yandex.net/c1cf9018843395a9bb6330116b34ca57d56831b0b17a6302c8e0d12a94061843.png" width="450"></td>
      <td>  </td>
     <td><img src="https://sbs.s3.yandex.net/ec79411d628a9380ff961eb94389326eee5433925cd387314f160a7d128cfed9.png" width="450"></td>
  </tr>
</table>

Стартуем оба пула и идем пить кофе

In [452]:
run_toloka_pool(msk_pool)
run_toloka_pool(spb_pool)

Pool 8808652 was started
Pool 8808656 was started


Выводы:

1. Аудитория из Санкт-Петербурга чаще выбираем вариант с "Шаверма". 

2. Аудитория из Москвы чаще выбираем вариант с "Шаурма". 

In [548]:
msk_results = pd.DataFrame(get_pool_completed_tasks(msk_pool)["winner_url"].value_counts()).reset_index()
msk_results["winner_url"] = msk_results["winner_url"] / sum(msk_results["winner_url"])

msk_results.columns = ["url", "rate"]
msk_results["url"] = msk_results["url"].apply(
                        lambda x: '<a href="{}"><img src="{}" width=300px></a>'.format(x, x))

pd.options.display.max_colwidth = -1
html_all = msk_results.to_html(escape=False, )
display(HTML(html_all))
pd.options.display.max_colwidth = 50


,url,rate
0,,0.78
1,,0.22


In [550]:
spb_results = pd.DataFrame(get_pool_completed_tasks(spb_pool)["winner_url"].value_counts()).reset_index()
spb_results["winner_url"] = spb_results["winner_url"] / sum(spb_results["winner_url"])

spb_results.columns = ["url", "rate"]
spb_results["url"] = spb_results["url"].apply(
                        lambda x: '<a href="{}"><img src="{}" width=300px></a>'.format(x, x))

pd.options.display.max_colwidth = -1
html_all = spb_results.to_html(escape=False, )
display(HTML(html_all))
pd.options.display.max_colwidth = 50


,url,rate
0,,0.54
1,,0.46


# Сравнение поисковых выдач



Есть несколько картинок:


<table>
  <tr>
<td>Запрос</td>
      <td>  </td>
     <td>Первый скриншот</td>
      <td>  </td>
     <td>Второй скриншот</td>
  </tr>
    <td>Дрель</td>
      <td>  </td>
     <td><img src="https://sbs.s3.yandex.net/059f49c93fce807412d023bb55c4844e36eb6db8ae587b9a34c11c3890762a7e.png" width="450"></td>
      <td>  </td>
     <td><img src="https://sbs.s3.yandex.net/1af370c9ad02557b5172b52176f37d1dbd9577359966b6daf0c6cfdc96dee955.png" width="450"></td>
  </tr>
    
  <tr>
    <td>Пассатижи</td>
      <td>  </td>
     <td><img src="https://sbs.s3.yandex.net/4d85b1a25fe8ec7872ae0b4265dc8299c74e72a2bbf5eb54c6300922f91f523e.png" width="450"></td>
      <td>  </td>
     <td><img src="https://sbs.s3.yandex.net/0998ab88cbe8113f3cf8d2f59c9b9730fd1e0ed0c0bfbcc7b0c739474d026de6.png" width="450"></td>
  </tr>
    
  <tr>    
     <td>Обои</td>
      <td>  </td>
     <td><img src="https://sbs.s3.yandex.net/d5749d7cb9b3e3fdb13a6d324d89a0a3b2bee0aa54f414148f62a2bc10a44bce.png" width="450"></td>
      <td>  </td>
     <td><img src="https://sbs.s3.yandex.net/cfb6cf74a996059e09a54b05c3bd3eedc6db5e160dcd9f8fac77552d3047b7b1.png" width="450"></td>
  </tr>
</table>

In [551]:
queries = [
    {
        "query": "Дрель",
        "first_url": "https://sbs.s3.yandex.net/059f49c93fce807412d023bb55c4844e36eb6db8ae587b9a34c11c3890762a7e.png", 
        "second_url": "https://sbs.s3.yandex.net/1af370c9ad02557b5172b52176f37d1dbd9577359966b6daf0c6cfdc96dee955.png"
    }, 
    {
        "query": "Пассатижи",
        "first_url": "https://sbs.s3.yandex.net/4d85b1a25fe8ec7872ae0b4265dc8299c74e72a2bbf5eb54c6300922f91f523e.png", 
        "second_url": "https://sbs.s3.yandex.net/0998ab88cbe8113f3cf8d2f59c9b9730fd1e0ed0c0bfbcc7b0c739474d026de6.png"
    }, 
    {
        "query": "Обои",
        "first_url": "https://sbs.s3.yandex.net/d5749d7cb9b3e3fdb13a6d324d89a0a3b2bee0aa54f414148f62a2bc10a44bce.png", 
        "second_url": "https://sbs.s3.yandex.net/cfb6cf74a996059e09a54b05c3bd3eedc6db5e160dcd9f8fac77552d3047b7b1.png"
    }, 
    
]

In [553]:
leroy_pool = clone_toloka_pool(template_pool_id)
print_toloka_pool_link(leroy_pool)

Pool was cloned. New pool was created. New pool id:  8811095
https://toloka.yandex.ru/requester/pool/8811095


In [558]:
leroy_tasks = []
for query in queries:
    leroy_tasks += prepare_toloka_task_for_2_screens(query["first_url"], query["second_url"], 
                                      default_question="Какая выдача больше подходит по запросу {}".format(query["query"]), 
                                     overlap=10, pool_id=leroy_pool)
    
upload_toloka_tasks(leroy_tasks)

Проверяем, что залилось и запускаем

In [559]:
run_toloka_pool(leroy_pool)

Pool 8811095 was started


In [606]:
leroy_results = get_pool_completed_tasks(leroy_pool)
leroy_results.head()

,image_left,image_right,question,comment,result,winner_url,loser_url
0,https://sbs.s3.yandex.net/cfb6cf74a996059e09a5...,https://sbs.s3.yandex.net/d5749d7cb9b3e3fdb13a...,Какая выдача больше подходит по запросу Обои,NaN,RIGHT,https://sbs.s3.yandex.net/d5749d7cb9b3e3fdb13a...,https://sbs.s3.yandex.net/cfb6cf74a996059e09a5...
1,https://sbs.s3.yandex.net/059f49c93fce807412d0...,https://sbs.s3.yandex.net/1af370c9ad02557b5172...,Какая выдача больше подходит по запросу Дрель,Вариант справа выглядит лучше по дизайну,RIGHT,https://sbs.s3.yandex.net/1af370c9ad02557b5172...,https://sbs.s3.yandex.net/059f49c93fce807412d0...
2,https://sbs.s3.yandex.net/4d85b1a25fe8ec7872ae...,https://sbs.s3.yandex.net/0998ab88cbe8113f3cf8...,Какая выдача больше подходит по запросу Пассатижи,шире выбор товара который запрашивают,LEFT,https://sbs.s3.yandex.net/4d85b1a25fe8ec7872ae...,https://sbs.s3.yandex.net/0998ab88cbe8113f3cf8...
3,https://sbs.s3.yandex.net/d5749d7cb9b3e3fdb13a...,https://sbs.s3.yandex.net/cfb6cf74a996059e09a5...,Какая выдача больше подходит по запросу Обои,NaN,RIGHT,https://sbs.s3.yandex.net/cfb6cf74a996059e09a5...,https://sbs.s3.yandex.net/d5749d7cb9b3e3fdb13a...
4,https://sbs.s3.yandex.net/cfb6cf74a996059e09a5...,https://sbs.s3.yandex.net/d5749d7cb9b3e3fdb13a...,Какая выдача больше подходит по запросу Обои,NaN,LEFT,https://sbs.s3.yandex.net/cfb6cf74a996059e09a5...,https://sbs.s3.yandex.net/d5749d7cb9b3e3fdb13a...


In [607]:
leroy_results = leroy_results.groupby(["question", "winner_url"])[["winner_url"]].count()
leroy_results.columns = ["wins_number"]
leroy_results = leroy_results.reset_index()

In [608]:
leroy_results

leroy_results["winner_url"] = leroy_results["winner_url"].apply(
                        lambda x: '<a href="{}"><img src="{}" width=300px></a>'.format(x, x))

pd.options.display.max_colwidth = -1
html_all = leroy_results.to_html(escape=False, )
display(HTML(html_all))
pd.options.display.max_colwidth = 50


,question,winner_url,wins_number
0,Какая выдача больше подходит по запросу Дрель,,7
1,Какая выдача больше подходит по запросу Дрель,,3
2,Какая выдача больше подходит по запросу Обои,,5
3,Какая выдача больше подходит по запросу Обои,,5
4,Какая выдача больше подходит по запросу Пассатижи,,7
5,Какая выдача больше подходит по запросу Пассатижи,,3


# SBSить можно не только картинки, но и видео.

In [35]:
first_video = """<iframe width="500" height="315" src="https://www.youtube.com/embed/2tSHYKbKAX8" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>"""
second_video = """<iframe width="500" height="315" src="https://www.youtube.com/embed/cbq3BoEZc9s" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>"""

videos_to_sbs = pd.DataFrame([first_video, second_video]).T

pd.options.display.max_colwidth = 20
html_all = videos_to_sbs.to_html(escape=False, )
display(HTML(html_all))
pd.options.display.max_colwidth = 50

,0,1
0,,


In [622]:
project_template = json.loads(open("project_templates/youtube_video_comparison_project_template.json").read())
project_id = create_toloka_project(project_template)


pool_template = json.loads(open("project_templates/pool_params_template.json").read())
template_video_pool_id = create_toloka_pool(project_id, pool_template)

New project was created. New project id:  29702
New pool was created. New pool id:  8812324


In [623]:
two_videos_pool_id = clone_toloka_pool(template_video_pool_id)


Pool was cloned. New pool was created. New pool id:  8812327


In [625]:
two_videos_pool_id = clone_toloka_pool(template_video_pool_id)


video_tasks = prepare_toloka_task_for_2_screens(first_video, second_video, 
                                      default_question="Какое видео с Яндекс.Алисой вам больше нравится?", 
                                     overlap=100, pool_id=two_videos_pool_id)
    
upload_toloka_tasks(video_tasks)
print_toloka_pool_link(two_videos_pool_id)

Pool was cloned. New pool was created. New pool id:  8812342
https://toloka.yandex.ru/requester/pool/8812342


Снова проверяем и запускаем

In [626]:
run_toloka_pool(two_videos_pool_id)

Pool 8812342 was started


In [635]:
two_videos_pool_id

'8812342'

In [638]:
video_results = pd.DataFrame(get_pool_completed_tasks(two_videos_pool_id)["winner_url"].value_counts()).reset_index()
video_results["winner_url"] = video_results["winner_url"] / sum(video_results["winner_url"])

video_results.columns = ["url", "rate"]

pd.options.display.max_colwidth = -1
html_all = video_results.to_html(escape=False, )
display(HTML(html_all))
pd.options.display.max_colwidth = 50


,url,rate
0,,0.74
1,,0.26


# Добавление вебвизора в проект

Можно записывать действия толокера с помощью вебвизора. Для этого нужно разобраться с счетчиком яндекс.метрики. 
https://yandex.ru/support/metrica/ и добавить его к себе в проект

In [14]:
project_for_webvisor = json.loads(open("project_templates/pictures_comparison_project_template.json").read())
project_for_webvisor = create_toloka_project(project_for_webvisor)


New project was created. New project id:  29735


In [17]:
pool_template = json.loads(open("project_templates/pool_params_template.json").read())
template_webvisor_pool_id = create_toloka_pool(project_for_webvisor, pool_template)

New pool was created. New pool id:  8835749


In [18]:
first_url = "https://sbs.s3.yandex.net/44aebb8c77a29efa25e8a5d1ac4eba9877b84efa1147b4b003b2a8380273cc73.png"
second_url = "https://sbs.s3.yandex.net/93656faf648029e3bf4bf18421b3dc08d85fe4bdd2683498cbf497f0487e89db.png"

print(first_url)
print(second_url)

https://sbs.s3.yandex.net/44aebb8c77a29efa25e8a5d1ac4eba9877b84efa1147b4b003b2a8380273cc73.png
https://sbs.s3.yandex.net/93656faf648029e3bf4bf18421b3dc08d85fe4bdd2683498cbf497f0487e89db.png


In [19]:
two_pictures_pool_id = clone_toloka_pool(template_webvisor_pool_id)
tasks = prepare_toloka_task_for_2_screens(first_url, second_url, 
                                         50, two_pictures_pool_id)


Pool was cloned. New pool was created. New pool id:  8835752


Далее зальем их в наш пул. 

In [20]:
upload_toloka_tasks(tasks)

In [21]:
print_toloka_pool_link(two_pictures_pool_id)

https://toloka.yandex.ru/requester/pool/8835752


In [22]:
run_toloka_pool(two_pictures_pool_id)

Pool 8835752 was started


Через некоторое время  в Яндекс.Метрике станут доступны вебвизорные записи!

<video width="700" controls="controls"  src="https://sbs.s3.yandex.net/9171f87e99eb0a5b581631b987d6dfc74fdb0a222d1479e9b774f6c0021f79b5/webvisor_example.mp4">

</video>